# Notebook to generate `functions` module

In [15]:
import inspect
import warnings

from pathlib import Path

from pprint import pformat

from mintalib import core

from inspect import Signature

from mintalib.samples import sample_prices
from mintalib.builder import annotate_parameter

class TEXT(str):
    def __repr__(self):
        return self

In [16]:

ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)



In [17]:

prices = sample_prices()


In [18]:
import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [19]:
import inspect

def first_param(func):
    return next(iter(inspect.signature(func).parameters), None)

def core_functions():
    return list(k for k, v in vars(core).items() if k.startswith("calc") and callable(v))

core_functions()

['calc_avgprice',
 'calc_typprice',
 'calc_wclprice',
 'calc_midprice',
 'calc_price',
 'calc_crossover',
 'calc_crossunder',
 'calc_flag',
 'calc_updown',
 'calc_sign',
 'calc_log',
 'calc_exp',
 'calc_diff',
 'calc_lag',
 'calc_min',
 'calc_max',
 'calc_sum',
 'calc_roc',
 'calc_mad',
 'calc_stdev',
 'calc_sma',
 'calc_ema',
 'calc_rma',
 'calc_wma',
 'calc_hma',
 'calc_dema',
 'calc_tema',
 'calc_ma',
 'calc_rsi',
 'calc_dmi',
 'calc_adx',
 'calc_pdi',
 'calc_mdi',
 'calc_trange',
 'calc_atr',
 'calc_natr',
 'calc_sar',
 'calc_cci',
 'calc_cmf',
 'calc_mfi',
 'calc_bop',
 'calc_bbands',
 'calc_keltner',
 'calc_ker',
 'calc_kama',
 'calc_macd',
 'calc_ppo',
 'calc_slope',
 'calc_rvalue',
 'calc_tsf',
 'calc_curve',
 'calc_stoch',
 'calc_streak',
 'calc_eval']

In [20]:
# Sample code for indicator

from mintalib.core import get_series, wrap_function, wrap_result

@wrap_function(core.calc_sma)
def sma(series, period: int, *, item: str = None):
    kwargs = dict(period=period)
    series = get_series(series, item=item)
    result = core.calc_sma(series, **kwargs)
    return wrap_result(result, series)



In [21]:
sma(prices, 20)

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    227.524000
2024-10-16    228.078500
2024-10-17    228.242500
2024-10-18    228.582500
2024-10-21    229.082999
Length: 11056, dtype: float64

In [22]:
PREAMBLE = '''"""
Calculation functions for technical analysis indicators.

The function names are all lower case and may conflict with standard functions,
so the best way to use this module is to alias it to a short name
like `ta` and access all functions as attributes.

The first parameter `series` or `prices` indicates whether the function
accepts a single series or a prices dataframe.

Functions that accept a series usually have an optional parameter `item`
to specify which column to use if the input is a dataframe.

All functions wrap their output to match the type of their input.

In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.
"""

# Do not edit! This file was generated

from . import core
from .core import get_series, wrap_function, wrap_result


nan = float('nan')

def __getattr__(name):
    if name.isupper() and name.lower() in globals():
        return globals()[name.lower()]
    raise AttributeError(f"Module {__name__} has no attribute {name!r}")

'''

def short_name(func):
    name = func.__name__
    if name.startswith("calc_"):
        name = name.removeprefix("calc_")
    else:
        raise ValueError("Expected a function names starting with 'calc_'")
    return name

def make_function(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = short_name(func)

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    newparams = [annotate_parameter(p) for p in parameters if p.name != "wrap"]

    if ftype == "series":
        newparams.append(inspect.Parameter("item", kind=inspect.Parameter.KEYWORD_ONLY, default=None, annotation=str))

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_function({qname})\n"
    code += f"def {name}{newsig}:\n"

    if ftype == "series":
        code += "    series = get_series(series, item=item)\n"

    kwargs = ", ".join(f"{p.name}={p.name}" for p in newparams[1:] if p.name not in ("series", "prices", "item"))
    code += f"    kwargs = dict({kwargs})\n"
    code += f"    result = {qname}({ftype}, **kwargs)\n"
    code += f"    return wrap_result(result, {ftype})\n"

    return code


TEXT(make_function(core.calc_ema))

@wrap_function(core.calc_ema)
def ema(series, period: int, *, adjust: bool = False, item: str = None):
    series = get_series(series, item=item)
    kwargs = dict(period=period, adjust=adjust)
    result = core.calc_ema(series, **kwargs)
    return wrap_result(result, series)

In [23]:
def test_function(func, name=None, verbose=False):
    code = make_function(func, name=name, preamble=True, verbose=verbose)
    return new_module("mintalib.functions", code)

test_function(core.calc_sma)


<module 'mintalib.functions'>

In [24]:

def make_functions(names=None):
    if names is None:
        names = core_functions()

    output = PREAMBLE

#    buffer = pformat(names, width=75, compact=True, indent=4)
#    buffer = buffer.replace("[", " ").replace("]", "")
#    output += f"__all__ = [\n{buffer}\n]\n"

    output += "__all__ = ()\n"
    output += "\n\n"

    for name in names:
        func = getattr(core, name)
        text = make_function(func)
        output += text + "\n\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"

    return output


code = make_functions()

TEXT(code[:1024] +  " ...")

"""
Calculation functions for technical analysis indicators.

The function names are all lower case and may conflict with standard functions,
so the best way to use this module is to alias it to a short name
like `ta` and access all functions as attributes.

The first parameter `series` or `prices` indicates whether the function
accepts a single series or a prices dataframe.

Functions that accept a series usually have an optional parameter `item`
to specify which column to use if the input is a dataframe.

All functions wrap their output to match the type of their input.

In particular the result of a function applied to a pandas series or dataframes
will have the same index as the input.
"""

# Do not edit! This file was generated

from . import core
from .core import get_series, wrap_function, wrap_result


nan = float('nan')

def __getattr__(name):
    if name.isupper() and name.lower() in globals():
        return globals()[name.lower()]
    raise AttributeError(f"Module {__name__

In [25]:
code = make_functions()

functions = new_module("mintalib.functions", code)

dir(functions)

['__all__',
 '__builtins__',
 '__doc__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'adx',
 'atr',
 'avgprice',
 'bbands',
 'bop',
 'cci',
 'cmf',
 'core',
 'crossover',
 'crossunder',
 'curve',
 'dema',
 'diff',
 'dmi',
 'ema',
 'eval',
 'exp',
 'flag',
 'get_series',
 'hma',
 'kama',
 'keltner',
 'ker',
 'lag',
 'log',
 'ma',
 'macd',
 'mad',
 'max',
 'mdi',
 'mfi',
 'midprice',
 'min',
 'nan',
 'natr',
 'pdi',
 'ppo',
 'price',
 'rma',
 'roc',
 'rsi',
 'rvalue',
 'sar',
 'sign',
 'slope',
 'sma',
 'stdev',
 'stoch',
 'streak',
 'sum',
 'tema',
 'trange',
 'tsf',
 'typprice',
 'updown',
 'wclprice',
 'wma',
 'wrap_function',
 'wrap_result']

In [26]:
assert callable(functions.sma)

res = functions.sma(prices, 20)

assert len(res) > 0


In [27]:
outfile = PKGDIR / "functions.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating functions.py ...


13482

In [28]:
from mintalib import functions

assert callable(functions.sma)

res = functions.sma(prices, 20)

assert len(res) > 0